In [1]:
#update libs
!pip install --upgrade pandas
!pip install --upgrade scikit-learn

Requirement already up-to-date: pandas in c:\anaconda\lib\site-packages (0.23.4)
  Found existing installation: scikit-learn 0.20.1
    Uninstalling scikit-learn-0.20.1:
      Successfully uninstalled scikit-learn-0.20.1


In [2]:
#import common libs
import numpy as np
import pandas as pd
import scipy.stats as st
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(font_scale=1.3)
from scipy import interp

In [6]:
#import special libs
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, Imputer,OrdinalEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import PowerTransformer,QuantileTransformer
from sklearn.preprocessing import label_binarize,MultiLabelBinarizer
from sklearn.pipeline import make_pipeline,Pipeline

from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

from itertools import cycle

from sklearn.multiclass import OneVsRestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,LinearSVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split 
from sklearn.model_selection import KFold,RandomizedSearchCV, cross_val_score

from sklearn.metrics import confusion_matrix, recall_score, brier_score_loss, roc_curve, auc
from sklearn.metrics import f1_score, make_scorer, roc_auc_score, precision_score
from sklearn.metrics import classification_report

from sklearn.decomposition import PCA, TruncatedSVD,IncrementalPCA

import glob

from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.calibration import CalibratedClassifierCV, calibration_curve


%matplotlib inline

In [43]:
import nltk

from nltk import word_tokenize
from nltk.stem.porter import PorterStemmer # one more stemmer? or lemmatisation
import re
from nltk.corpus import stopwords

seed = 321

Загрузим датасет с классифицируемыми текстами.

In [8]:
from sklearn.datasets import fetch_20newsgroups

Итак, задача данной работы:



#### EDA.

Посмотрим на данные, для этого:
- скачаем датасет целиком
- посмотрим на категории в данных
- распределение документов по категориям
- посмотрим на примеры данных

In [47]:
# здесь shuffle = true, т.к. мы собираемся использовать SGDClassifier; Naive Bayes - они базируются на принципе iid данных 
#и чтобы не получить смещенную оценку
newsgroups_train = fetch_20newsgroups(subset='all', random_state=seed, shuffle=True)

from pprint import pprint
pprint(list(newsgroups_train.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [48]:
print("%d documents" % len(newsgroups_train.data))
print("%d categories" % len(newsgroups_train.target_names))

18846 documents
20 categories


Посмотрим на образец данных.

In [49]:
newsgroups_train.data[15]

'From: Jim_Johnson@abcd.houghton.mi.us (Jim Johnson)\nSubject: Run box w/o cover ??\nOrganization: Amiga BitSwap Central Dispatch\nLines: 25\n\n\nB(>i am interested in getting the pulse of this group regarding\nB(>extended operation of my G2K 486-33V with the cover removed\nB(>from the enclosure.  there are a # of reasons i am considering\nB(>this, including quick access to jumpers during complex i/o card\nB(>setups.\n\nB(>my concern is that without a complete enclosure to direct the\nB(>cooling flow of air from the fan, "hot spots" may develop on my\nB(>motherboard or elsewhere.\n\nIf you have an adequate supply of air moving over the system (most\noffices or homes have positive ventilation) you can generally run a\nsystem without the cover for extended periods without a problem. (I\'m\ntalking about completely removing the cover - not just leaving the slots\nuncovered.) HOWEVER, the biggest reason you have a cover to begin with\nis RF sheilding. Operating a system without the full co

In [42]:
newsgroups_train.target[15]

17

#### Data preprocessing.

Предобработаем данные, для этого:
- отсечем стоп слова (уберем символы и комбинации, которые не составляют смысловой нагрузки).
- 